
# DQN Training for Construction Logistics Ordering System

This notebook contains the core implementation of the DQN model used in the paper.  
It includes the environment definition, DQN architecture, training loop, and policy extraction logic.  
All parameters and reward structures align with those described in Section 3 of the manuscript.
